In [1]:
# Convolutional neural network from scratch 
import cv2
import numpy as np 
import pandas as pd 
from matplotlib import pyplot as plt

In [2]:
data = pd.read_csv('mnist_train.csv')
data.head()

,label,1x1,1x2,1x3,1x4,1x5,1x6,1x7,1x8,1x9,...,28x19,28x20,28x21,28x22,28x23,28x24,28x25,28x26,28x27,28x28
0,5,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,4,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,9,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [3]:
data = np.array(data) # transform our csv into array 
lines, cols = data.shape # lines = 42000 / columns = 785 bcz the images are 28x28=784 pixels +1 with the beggining cols of label
np.random.shuffle(data) # because the train data are possibly sorted with a sequences of 0 then 1 then 2 and so on 
# we shuffe the dataset in order to avoid overfitting
data_validation = data[0:1000].T # Data transposition is performed in this particular context to facilitate 
# the manipulation of examples and features. Some machine learning algorithms and libraries prefer 
# this matrix representation with examples in columns rather than rows.
# data validation variable is used to evaluate and confirm the performance of our future model while training
label_val = data_validation[0]
result_val = data_validation[1:cols]

data_train = data[1000:lines].T
label_train = data_train[0] # we did the transpose so the first line is compose only by label
result_train = data_train[1:cols]

In [78]:
def init_params() :
    # W => weights / b => bias ... In the beginning both are generated randomly 
    # 1st param : nb neurons on the actual layer / 2st param : nb neurons of the previous layer
    neuronsLayerOne = 10 
    neuronsLayerTwo = 10
    W1 = np.random.rand(neuronsLayerOne,cols-1) # -1 => to understand refer to what we said on the line of creation of cols
    b1 = np.random.rand(neuronsLayerOne,1) # which means for each neurons we generate a random number which will corresponding to the bias
    # bias doesn't have to be necessary between 0 and 1 ! 
    W2 = np.random.rand(neuronsLayerTwo,neuronsLayerOne)
    b2 = np.random.rand(neuronsLayerTwo,1)
    return W1, b1, W2, b2
def ReLu(Z) : #activation function non linear 
    return np.maximum(0,Z)
def softmax(Z) : 
    return np.exp(Z)/sum(np.exp(Z))
def forward_propagation(W1, b1, W2, b2, X) : # X is the input data (pixels 1 image) / Z -> unactivated layer / A -> activated 
    # forward propagation is when you take images and run them through the CNN / It's a linear transformation
    Z1 = W1.dot(X)+b1 # X -> several lines and 1 column due to the transpose ! (l,c) x (l,c)  W1 -> l : 1 c : 10 
    # we tend to represent layer of CNN in line but we do not need to adapt this in our line of code bcz it's not important
    A1 = ReLu(Z1)
    Z2 = W2.dot(X)+b2
    A2 = softmax(Z2) # final result 
    # final result Y of our CNN will be score associated to each prediction such as this
    # [0 0.05 0.7 0.01 0.09 0.1 0 0.05 0 0] = total equals to 1 such as a distribution function ! 
    return Z1, A1, Z2, A2
def one_hot(Y) : # function that will take the label of our data and for each label transform it into an encoded matrix 
    # example : 0 : 100000000 / 4 : 0000100000 / 9 : 0000000001 ...
    labelsNb = 10 # 0 1 2 3 4 5 6 7 8 9
    one_hot_Y = np.zeros((Y.size, labelsNb)) # to specify tuple we put () / Y.size => number of values of labels that we pass to the function 
    one_hot_Y[np.arange(Y.size),Y] = 1 
    one_hot_Y = one_hot_Y.T
    return one_hot_Y
def backward_propagation(Z1, A1, Z2, A2, W2, X, Y) :
    m = Y.size # equals line
    one_hot_Y = one_hot(Y)
    dZ2 = A2 - one_hot_Y
    dW2 = 1 / m * dZ2.dot(A1.T)
    db2 = 1 / m * np.sum(dZ2)
    dZ1 = W2.T.dot(dZ2) * deriv_ReLU(Z1) #TO define 
    dW1 = 1 / m * dZ1.dot(X.T)
    db1 = 1 / m * np.sum(dZ1)
    return dW1, db1